# Using SQL queries in `pandas`

You can create a SQL database in Python with the `sqlalchemy` library.

You can also run SQL *queries* in Python on `pandas` dataframes using the `pandasql` library, which is called `ps` while importing in the same way that `pandas` is called `pd`. This needs installing first in Colab.

In [23]:
import pandas as pd
#this library is needed to create a sql database
from sqlalchemy import *
#or there's this to do sql queries
!pip install pandasql
import pandasql as ps

## Using SQLalchemy

The [documentation for SQLalchemy](https://docs.sqlalchemy.org/en/14/core/metadata.html) explains how to create tables for the SQL database.

The best way to think about this is that you need a SQL *engine*, a database *structure* (an object, in Python terms) and within that *tables*, so there's a whole infrastructure to create.

In [4]:
#create a SQL engine
engine = create_engine('sqlite:///:memory:')

In [5]:
#create an object that will contain the database features
metadata_obj = MetaData()

In [6]:
#define a table in the SQL database using that
user = Table('user', metadata_obj,
    Column('user_id', Integer, primary_key=True),
    Column('user_name', String(16), nullable=False),
    Column('email_address', String(60)),
    Column('nickname', String(50), nullable=False)
)

In [7]:
#create that table (or rather, all items so far defined)
metadata_obj.create_all(engine)

## Reading from SQL databases using `pandas`

You can read from a SQL database using `pandas`'s [`read_sql` function](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html). This needs at least two ingredients: 

* a SQL query as a string, and 
* a "connection"

Normally you are connecting to remote SQL server - but if you've set it up locally using `sqlalchemy` then you specify the name of the 'engine' - which we called `engine` earlier on.

In [8]:
#run a SQL query on that table 'user'
userdf = pd.read_sql("SELECT * FROM user;", engine)
userdf

,user_id,user_name,email_address,nickname


In [16]:
#append a row - note it will fill unspecified columns with NaN
userdf.append({"user_id":"1"}, ignore_index=True)

,user_id,user_name,email_address,nickname
0,1,NaN,NaN,NaN


In [17]:
#append another row - note that the previous line didn't change the dataframe - it just showed you what it would look like
userdf.append({"user_id":"1", "user_name":"Paul","email_address":"paul.bradshaw@bcu.ac.uk","nickname":"Him"}, ignore_index=True)

,user_id,user_name,email_address,nickname
0,1,Paul,paul.bradshaw@bcu.ac.uk,Him


In [18]:
#this time run both lines of code, but in a different order, and each time assign the results to the original pandas dataframe
userdf = userdf.append({"user_id":"1", "user_name":"Paul","email_address":"paul.bradshaw@bcu.ac.uk","nickname":"Him"}, ignore_index=True)
userdf = userdf.append({"user_id":"1"}, ignore_index=True)
userdf

,user_id,user_name,email_address,nickname
0,1,Paul,paul.bradshaw@bcu.ac.uk,Him
1,1,NaN,NaN,NaN


## Using `pandasql`

This library allows you to query a pandas dataframe using SQL queries.

We've generated a URL for a CSV of COVID data from the [UK Coronavirus Dashboard data download page](https://coronavirus.data.gov.uk/details/download)

In [1]:
covidcsvurl = "https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&metric=newCasesBySpecimenDateRollingSum&metric=newCasesBySpecimenDateRollingRate&metric=newCasesBySpecimenDateChange&metric=newCasesBySpecimenDateChangePercentage&metric=newCasesBySpecimenDateDirection&format=csv"

In [10]:
testdata = pd.read_csv(covidcsvurl)
testdata.head()

,regionCode,regionName,UtlaCode,UtlaName,LtlaCode,LtlaName,areaCode,areaName,areaType,date,newCasesBySpecimenDateChange,newCasesBySpecimenDateChangePercentage,newCasesBySpecimenDateDirection,newCasesBySpecimenDateRollingRate,newCasesBySpecimenDateRollingSum
0,E12000001,North East,E08000037,Gateshead,E08000037,Gateshead,E02001682,Ryton,msoa,2021-11-16,-2.0,-4.7,DOWN,487.2,41
1,E12000001,North East,E08000037,Gateshead,E08000037,Gateshead,E02001683,Crawcrook & Greenside,msoa,2021-11-16,-16.0,-25.8,DOWN,506.0,46
2,E12000001,North East,E08000037,Gateshead,E08000037,Gateshead,E02001684,"Pelaw, Felling Shore & Bill Quay",msoa,2021-11-16,-6.0,-28.6,DOWN,222.6,15
3,E12000001,North East,E08000037,Gateshead,E08000037,Gateshead,E02001685,Blaydon South,msoa,2021-11-16,-3.0,-9.1,DOWN,370.1,30
4,E12000001,North East,E08000037,Gateshead,E08000037,Gateshead,E02001686,Blaydon North & Winlaton Mill,msoa,2021-11-16,10.0,52.6,UP,435.5,29


In [14]:
len(testdata)

377217

In [11]:
testdata.columns

Index(['regionCode', 'regionName', 'UtlaCode', 'UtlaName', 'LtlaCode',
       'LtlaName', 'areaCode', 'areaName', 'areaType', 'date',
       'newCasesBySpecimenDateChange',
       'newCasesBySpecimenDateChangePercentage',
       'newCasesBySpecimenDateDirection', 'newCasesBySpecimenDateRollingRate',
       'newCasesBySpecimenDateRollingSum'],
      dtype='object')

In [20]:
testdata.dtypes

regionCode                                 object
regionName                                 object
UtlaCode                                   object
UtlaName                                   object
LtlaCode                                   object
LtlaName                                   object
areaCode                                   object
areaName                                   object
areaType                                   object
date                                       object
newCasesBySpecimenDateChange              float64
newCasesBySpecimenDateChangePercentage    float64
newCasesBySpecimenDateDirection            object
newCasesBySpecimenDateRollingRate         float64
newCasesBySpecimenDateRollingSum            int64
dtype: object

In [19]:
#show just West Midlands and sum cases
ps.sqldf("SELECT regionName, newCasesBySpecimenDateRollingSum FROM testdata WHERE regionName == 'West Midlands'")

,regionName,newCasesBySpecimenDateRollingSum
0,West Midlands,36
1,West Midlands,29
2,West Midlands,59
3,West Midlands,39
4,West Midlands,59
...,...,...
41934,West Midlands,4
41935,West Midlands,4
41936,West Midlands,6
41937,West Midlands,5


In [22]:
#store the west midlands filter
wmonly = ps.sqldf("SELECT regionName, newCasesBySpecimenDateRollingSum FROM testdata WHERE regionName == 'West Midlands'")
len(wmonly)

41939